# HDMR/Sobol Expansion Terms
## Definitions

Given a function $f : [0, 1]^d \rightarrow \mathbb{R}$ where $d = 3$

$$
f(\vec{Z}) = Z_1 + Z_2^2 + Z_1 Z_2 + Z_2 Z_3^2,
$$

and $Z_i \sim \mathcal{U}[0, 1]\ \forall i \in \{1, 2, 3 \}$, show
the Sobol representation of this function. 

The general Sobol representation is given by 

$$
\begin{aligned}
f_0 &= \int_\Gamma f(z)\ dz \\
f_i(z_i) &= \int_{\Gamma^{d-1}} f(z)\ dz_{\sim i} - f_0 \\
f_{ij}(z_i, z_j) &= \int_{\Gamma^{d-2}} f(z)\ dz_{\sim ij} - f_i(z_i) - f_j(z_j) - f_0
\end{aligned}
$$

on the domain $\Gamma$, in this case $\Gamma = [0, 1]^3$ and the notation $\{\sim i\}$
denotes all indices **except** i. 

Then the Sobol representation for our problem is 

$$
\begin{aligned}
f_0(z_0) &= \int_0^1 \int_0^1 \int_0^1 f(z)\ dz, \\
f_1(z_1) &= \int_0^1 \int_0^1 f(z)\ dz_2\ dz_3 - f_0, \\
f_2(z_2) &= \int_0^1 \int_0^1 f(z)\ dz_1\ dz_3 - f_0, \\
f_3(z_3) &= \int_0^1 \int_0^1 f(z)\ dz_1\ dz_2 - f_0, \\
f_{12}(z_1, z_2) &= \int_0^1 f(z)\ dz_3 - f_1(z_1) - f_2(z_2) - f_0 \\
f_{13}(z_1, z_3) &= \int_0^1 f(z)\ dz_2 - f_1(z_1) - f_3(z_3) - f_0 \\
f_{23}(z_2, z_3) &= \int_0^1 f(z)\ dz_1 - f_2(z_2) - f_3(z_3) - f_0
\end{aligned}
$$
